In [1]:
#import dependencies
from pybaseball import team_pitching
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

from pybaseball import team_batting
from pybaseball import schedule_and_record
from pybaseball import bwar_bat
import csv
import os
import json
import requests

In [2]:
#MySQL connection
engine = create_engine('mysql+pymysql://root:<yourpassword>$@localhost:3306/baseball_data')

#function to import data in Mysql
def import_in_sql(data, table):
    data.to_sql(name=table, con=engine, if_exists='replace', index=True)

In [3]:
#example of import data in to Mysql
#convert data in to panda dataframe
pitching_data = team_pitching(2015,2016)
pitching_data.head()
data = pd.DataFrame(data = pitching_data)
#clean data
pitching_data_sql = data[['Season','K/9']]
pitching_data_sql.rename(columns={'K/9':'K_9'}, inplace=True)
#import data in to MySQL
import_in_sql(data=pitching_data_sql, table='pitching_data_table_n')


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [4]:
##Creating team data from 2015-2018
##Displaying the data
team_results_df = team_batting(2015, 2018)
team_results_df.head()
team_results_df[["Team", "Season", "SB"]]
team_results_sql=team_results_df[["Team", "Season", "SB"]]
#import data in to MySQL
import_in_sql(data=team_results_sql, table='team_batting')

In [5]:
##Creating team list with abbreviations 
##Displaying the data
file = os.path.join("baseball.csv")
df = pd.read_csv(file, encoding="ISO-8859-1")
df.head()

,TeamName,ABBR,HomeBase
0,Angels,LAA,Los Angeles
1,Astros,HOU,Houston
2,Athletics,OAK,Oakland
3,BlueJays,TOR,Toronto
4,Braves,ATL,Atlanta


In [6]:
##Creating schedule data
##Displaying the data
schedule_data = schedule_and_record(2018, "KCR")
schedule_data.head()

,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled
1,"Thursday, Mar 29",KCR,Home,CHW,L,7.0,14.0,9.0,0-1,3.0,1.0,Shields,Duffy,None,3:26,D,36517.0,-1,None
2,"Saturday, Mar 31",KCR,Home,CHW,L,3.0,4.0,9.0,0-2,5.0,2.0,Farquhar,Maurer,Soria,2:54,N,17564.0,-2,None
3,"Monday, Apr 2",KCR,@,DET,L,1.0,6.0,9.0,0-3,5.0,2.0,Liriano,Hammel,None,2:46,D,15476.0,-3,None
4,"Tuesday, Apr 3",KCR,@,DET,W,1.0,0.0,9.0,1-3,4.0,1.0,Junis,Boyd,Herrera,2:17,D,15083.0,1,None
5,"Friday, Apr 6",KCR,@,CLE,L,2.0,3.0,9.0,1-4,5.0,2.5,Carrasco,Duffy,Allen,2:41,D,34720.0,-1,None


In [7]:
##Creating short schedule data
##Displaying the data
schedule_data_short = schedule_data.loc[:,["Tm", "Home_Away", "Opp","W/L", "Time"]]
schedule_data_short.head()

,Tm,Home_Away,Opp,W/L,Time
1,KCR,Home,CHW,L,3:26
2,KCR,Home,CHW,L,2:54
3,KCR,@,DET,L,2:46
4,KCR,@,DET,W,2:17
5,KCR,@,CLE,L,2:41


In [8]:
##Creating lists to use in for loop
abbr_list= df['ABBR'].tolist()
city_list= df['HomeBase'].tolist()

In [9]:
#Sorting data
wh_df = schedule_data_short.loc[(schedule_data_short["Home_Away"] == "Home")&(schedule_data_short["W/L"] == "W")].count()
lh_df = schedule_data_short.loc[(schedule_data_short["Home_Away"] == "Home")&(schedule_data_short["W/L"] == "L")].count()

In [10]:
#using for loop to get number of winning and lossing games
win_num_list=[]
loss_num_list = []
for x in abbr_list:
    schedule_data = schedule_and_record(2017, x)
    schedule_data_short = schedule_data.loc[:,["Tm", "Home_Away", "Opp","W/L", "Time"]]
    wh_df = schedule_data_short.loc[(schedule_data_short["Home_Away"] == "Home")&(schedule_data_short["W/L"] == "W")]
    lh_df = schedule_data_short.loc[(schedule_data_short["Home_Away"] == "Home")&(schedule_data_short["W/L"] == "L")]
    win_num = len(wh_df['Tm'])
    loss_num = len(lh_df['Tm'])
    win_num_list.append(win_num)
    loss_num_list.append(loss_num)

In [11]:
api_key = "25bc90a1196e6f153eece0bc0b0fc9eb"

In [12]:
#using json request to get coordinates for each city
url = "http://api.openweathermap.org/data/2.5/weather?"
lat = []
lon = []

for city in city_list:
# Build query URL
    query_url = url + "appid=" + api_key + "&q=" + city
    response = requests.get(query_url).json()
    lat.append(response['coord']['lat'])
    lon.append(response['coord']['lon'])
    

In [13]:
#creating data frame with all information
overview = pd.DataFrame({"Team": abbr_list,
                         "Win": win_num_list,
                         "Loss": loss_num_list,
                        "lat": lat,
                        "lon": lon})

In [14]:
overview.to_csv('baseball_coordinates.csv')
overview.to_json('baseball_coordinates.json', orient='records')

In [15]:
#import data in to MySQL
import_in_sql(data=overview, table='schedule_and_record')

In [17]:
#importing and cleaning bwar_bat data
bwar_bat_df = bwar_bat()
bwar_bat_df.sort_values(by=['year_ID'])
bwar_bat_df_new = bwar_bat_df[["name_common","player_ID","year_ID","team_ID","salary","PA","G"]]
bwar_bat_df_new = bwar_bat_df_new.replace({'NaN': None})
bwar_bat_df_new = bwar_bat_df_new[bwar_bat_df_new.salary.notna()]
bwar_bat_df_new = bwar_bat_df_new[bwar_bat_df_new.PA.notna()]
bwar_bat_df_new = bwar_bat_df_new[bwar_bat_df_new.G.notna()]

#import bwar_bat data in to MySQL
import_in_sql(data=bwar_bat_df_new, table='bwar_bat')